## Browse DKRZ catalog on Levante HPC system

Search for LUMIP simulations and available models

In [5]:
# import intake
import os
import glob
import netCDF4 as nc

import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy as cart
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point

import numpy as np
import xarray as xr
import cftime
import pandas as pd

# Regridding
import xesmf as xe


In [6]:
# # Multiprocessing
# import multiprocessing
# from dask.distributed import Client, progress, wait # Libaray to orchestrate distributed resources
# from dask_jobqueue import SLURMCluster

# ncpu = multiprocessing.cpu_count()
# processes = False
# nworker = 1
# threads = ncpu // nworker
# print(
#     f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}",
# )
# client = Client(
#     processes=processes,
#     threads_per_worker=threads,
#     n_workers=nworker,
#     memory_limit="16GB",
# )
# client

In [12]:
model = "CESM2"
data_path = os.path.join("F:/Data/LUMIP/", model)
out_path = "G:/My Drive/MPIM/data/"

# Functions

In [8]:
def xr_fix_time(xr_in, date_start, date_end):
    
    if xr_in['time'].dt.calendar == 'noleap' or xr_in['time'].dt.calendar == '360_day':
        xr_in = xr_in.convert_calendar(calendar = 'gregorian', align_on = 'date')
        
    else: None
    
    time = pd.date_range(start=date_start, end=date_end, freq='M').to_numpy(dtype='datetime64[ns]')
    xr_out = xr_in.assign_coords(time = time)
    
    return xr_out

In [9]:
def xr_clean(xr_in, dims):
    data = xr_in.copy()
    for i,d in enumerate(dims):
        if d in data.coords and d in data.dims:
            data = data.drop(d)
            data = data.isel({d : 0})
        if d in data.coords:
            data = data.drop(d)
        if d in data.dims and d not in data.coords:
            data = data.isel({d : 0})
        if data.attrs == {}:
            None
        else:
            if d in data.data_vars:
                data = data.drop_vars(d)
    xr_out = data
    return xr_out

In [10]:
def lon180(ds):
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
    return ds

In [11]:
import xesmf as xe

def xr_regrid(data, method, lon_bnds, lat_bnds, xres, yres):
    # Regrid XARRAY using xESMF library (https://xesmf.readthedocs.io/en/stable/index.html)
    # data: input data
    # method: "bilinear", "conservative", "conservative_normed", "patch", "nearest_s2d", "nearest_d2s"
    
    lonmin = lon_bnds[0]; lonmax = lon_bnds[1]
    latmin = lat_bnds[0]; latmax = lat_bnds[1]
    xr_out = xe.util.cf_grid_2d(lonmin, lonmax, xres, latmin, latmax, yres)
   
    regrid = xe.Regridder(data, xr_out, method)

    data_regrid = regrid(data, keep_attrs=True)

    return data_regrid

# PFT data

In [8]:
variables = ["treeFrac"]
members = ["r1i1p1f1"]

# IPSL has also cropFracC4

In [26]:
# data_path = '/home/m/m301093/data/new_models/pft'     

scenario = 'ssp370-ssp126Lu'
xr_aff_esm_pft = []
filepath = glob.glob(os.path.join(data_path + '/' + variables + '_*_' + models + "_" + scenario + '_*'))[0]       ## List of files sorted by name
xr_aff_esm_pft = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
# xr_aff_esm_pft.append(content)
# xr_aff_esm_pft.append(xr.merge(xr_aff, compat = "override"))


scenario = 'ssp370'
xr_ctl_esm_pft = []
filepath = glob.glob(os.path.join(data_path + '/' + variables + '_*_' + models + "_" + scenario + '_*'))[0]       ## List of files sorted by name
xr_ctl_esm_pft = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
# xr_ctl_esm_pft.append(content)
# xr_ctl_esm_pft.append(xr.merge(xr_ctl, compat = "override"))


Preprocessing

In [28]:
import dask
dask.config.set({"array.slicing.split_large_chunks": False})

xr_aff_esm_pft = xr_aff_esm_pft.sel(lat=slice(-60,90))
xr_ctl_esm_pft = xr_ctl_esm_pft.sel(lat=slice(-60,90))

xr_aff_esm_pft = xr_clean(xr_aff_esm_pft, ['dcpp_init_year', 'member_id', 'lon_bnds', 'lat_bnds', 'time_bnds', 'bnds'])
xr_ctl_esm_pft = xr_clean(xr_ctl_esm_pft, ['dcpp_init_year', 'member_id', 'lon_bnds', 'lat_bnds', 'time_bnds', 'bnds'])

xr_aff_esm_pft = lon180(xr_aff_esm_pft)
xr_ctl_esm_pft = lon180(xr_ctl_esm_pft)

xr_aff_esm_pft = xr_fix_time(xr_aff_esm_pft,"2015-01","2101-01")
xr_ctl_esm_pft = xr_fix_time(xr_ctl_esm_pft,"2015-01","2101-01")

Save data

In [31]:
xr_aff_esm_pft.to_netcdf(out_path + "/xr_ssp370_"+models+"_pft.nc")
xr_ctl_esm_pft.to_netcdf(out_path + "/xr_ssp126Lu_"+models+"_pft.nc")


## Climate data

In [32]:
models = ["CMCC-ESM2"] #["ACCESS-ESM1-5", "CMCC-ESM2", "IPSL-CM6A-LR"]

variables = ["tas", "ts", "pr", "mrso", "rlds", "rsds", "rsus", "hfls", "hfss", "huss", "ps"]
            #  [["tas", "ts", "pr", "rlds", "rsds", "rsus", "hfls", "hfss", "huss", "ps"],
            #  ["tas", "ts", "pr", "rlds", "rsds", "rsus", "hfls", "hfss", "huss", "ps"],
            #  ["tas", "ts", "pr", "rlds", "rsds", "rsus", "hfls", "hfss", "huss", "ps"]]
             
members = ["r1i1p1f1"] #[["r1i1p1f1"],
        #    ["r1i1p1f1"],
        #    ["r1i1p1f1"]]

# IPSL has also cropFracC4

In [74]:
import glob
import netCDF4

# data_path = '/home/m/m301093/data/new_models/climate'             

scenario = 'ssp370-ssp126Lu'
xr_aff_esm_clim = []
xr_aff = []
for v,vv in enumerate(variables): 
        filepath = glob.glob(os.path.join(data_path+'/' + vv + '_*_' + models[0] + "_" + scenario + '_*'))[0]                                       ## List of files sorted by name
        content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
        xr_aff.append(content)
        xr_aff_esm_clim = xr.merge(xr_aff, compat = "override")


scenario = 'ssp370'
xr_ctl_esm_clim = []
xr_ctl = []
for v,vv in enumerate(variables): 
        filepath = glob.glob(os.path.join(data_path+'/' + vv + '_*_' + models[0] + "_" + scenario + '_*'))[0]                                       ## List of files sorted by name
        content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
        xr_ctl.append(content)
        xr_ctl_esm_clim = xr.merge(xr_ctl, compat = "override")


In [63]:
xr_aff_esm_clim = xr_aff_esm_clim.sel(lat=slice(-60,90))
xr_ctl_esm_clim = xr_ctl_esm_clim.sel(lat=slice(-60,90))
# xr_hist_esm_clim = xr_hist_esm_clim.sel(lat=slice(-60,90))

xr_aff_esm_clim = xr_clean(xr_aff_esm_clim, ['dcpp_init_year', 'member_id', 'lon_bnds', 'lat_bnds', 'time_bnds', 'bnds'])
xr_ctl_esm_clim = xr_clean(xr_ctl_esm_clim, ['dcpp_init_year', 'member_id', 'lon_bnds', 'lat_bnds', 'time_bnds', 'bnds'])
# # xr_hist_esm_clim = xr_clean(xr_hist_esm_clim, ['dcpp_init_year', 'member_id', 'lon_bnds', 'lat_bnds', 'time_bnds', 'bnds'])

xr_aff_esm_clim = lon180(xr_aff_esm_clim)
xr_ctl_esm_clim = lon180(xr_ctl_esm_clim)
# # xr_hist_esm_clim = lon180(xr_hist_esm_clim)

xr_aff_esm_clim = xr_fix_time(xr_aff_esm_clim,"2015-01","2101-01")
xr_ctl_esm_clim = xr_fix_time(xr_ctl_esm_clim,"2015-01","2101-01")
# # xr_hist_esm_clim = xr_fix_time(xr_hist_esm_clim,"2015-01","2101-01")

In [69]:
xr_ctl_esm_clim.to_netcdf(out_path + "/xr_ssp370_"+models[0]+".nc")
xr_aff_esm_clim.to_netcdf(out_path + "/xr_ssp126Lu_"+models[0]+".nc")